<a href="https://colab.research.google.com/github/ktynski/Marketing_Automations_Notebooks_With_GPT/blob/main/Prompt_Chaining_Instant_Content_Plan_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


http://www.Frac.tl - My Agency, we earn press and massively improve the ranking potential of our client's domains through a data-journalism style approach to earned media. We are excited about the limitless possibilities of AI enabled processes in Content Marketing, SEO, and PR. Email me at Kristin@frac.tl if you would like to learn more about our services.

In [ ]:
!pip install openai
!pip install langchain
!pip install asyncio

In [ ]:

import pandas as pd
import openai
import re
import concurrent.futures

# Set up OpenAI API key
openai.api_key = "Enter your API Key"


def generate_article_title(topic, subtopic, subsubtopic):
    prompt = f"Give an SEO optimized an article title about'{subsubtopic}' under the topic of '{topic}' and the subtopic of '{subtopic}'"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=800,
        n=1,
        stop=None,
        temperature=0.7,
    )
    while not response.choices[0].text.strip():
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.7,
        )
    while not response.choices[0].text.strip():
        time.sleep(5)
    return response.choices[0].text.strip()

def generate_subsubtopics(topic, subtopic):
    prompt = f"List 10 SEO Optimized sub-subtopics under the topic of '{topic}' and the subtopic of '{subtopic}'"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=0.7,
    )
    while not response.choices[0].text.strip():
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=200,
            n=1,
            stop=None,
            temperature=0.7,
        )
    subsubtopics = re.findall(r"\d+\. (.*)", response.choices[0].text.strip())
    return subsubtopics

def generate_subtopics(topic):
    prompt = f"List 10 seo optimized subtopics under the topic of '{topic}'"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=0.7,
    )
    while not response.choices[0].text.strip():
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            max_tokens=200,
            n=1,
            stop=None,
            temperature=0.7,
        )
    subtopics = re.findall(r"\d+\. (.*)", response.choices[0].text.strip())
    return subtopics

def generate_data(topic):
    subtopics = generate_subtopics(topic)
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        subsubtopic_futures = {executor.submit(generate_subsubtopics, topic, subtopic): subtopic for subtopic in subtopics}
        for future in concurrent.futures.as_completed(subsubtopic_futures):
            subtopic = subsubtopic_futures[future]
            subsubtopics = future.result()
            for subsubtopic in subsubtopics:
                results.append((topic, subtopic, subsubtopic))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        article_title_futures = {executor.submit(generate_article_title, topic, subtopic, subsubtopic): (subtopic, subsubtopic) for (topic, subtopic, subsubtopic) in results}
        for future in concurrent.futures.as_completed(article_title_futures):
            (subtopic, subsubtopic) = article_title_futures[future]
            article_title = future.result()
            while not article_title:
                article_title = generate_article_title(topic, subtopic, subsubtopic)
            results.append((topic, subtopic, subsubtopic, article_title))

    df = pd.DataFrame(results, columns=["topic", "subtopic", "subsubtopic", "article_title"])
    return df

if __name__ == "__main__":
    topic = "Fishing"
    df = generate_data(topic)
    for i in range(200):
        article_title = generate_article_title(df.loc[i, 'topic'], df.loc[i, 'subtopic'], df.loc[i, 'subsubtopic'])
        df.loc[i, 'article_title'] = article_title
    print(df)

In [ ]:
df.to_csv("fishing.csv")